## Basic Multigrid Algorithm
*July 5th, 2024*

In [1]:
import numpy as np
import numpy.linalg as LA

In [2]:
def make_multigrid_v_cycle_1d_iter(A, f, Ns: list[int], relax):
    """
    Ns: List of subgrid panel counts (nodes = panels + 1) from high to small
    """
    Ns = iter(Ns)

    def make_restriction(m, n):
        """
        Example: 
        m = 6
        n = 2
        [1 0 0 0 0 0 0]
        [0 0 0 1 0 0 0]
        [0 0 0 0 0 0 1]
        """
        k = m // n
        stencil = np.zeros(k)
        stencil[0] = 1
        rows = [np.pad(stencil, (k * i, m - k * i), 'constant') for i in range(n + 1)]
        A = np.vstack(rows)
        
        return A[:,:m+1]

    def make_interpolation(m, n):
        """
        Example: 
        m = 6
        n = 2
        [1 2/3 1/3 0 0 0 0]T
        [0 1/3 2/3 1 0 0 0]
        [0 0 0 0 0 0 1]
        """
        k = m // n
        stencil = np.fromfunction(lambda i: np.array((1 - i / k, i / k)), shape = [k])
        R = np.zeros((n + 1, m + 1))
        for i in range(n):
            R[i:i+2, k * i:k*(i+1)] = stencil
        R[-1,-1] = 1
        return np.transpose(R)

    def v_cycle(A, u, b):
        # relax, restrict, v-cycle, interpolate and add, relax
        u = relax(A, u, b)
        r = b - np.matmul(A, u)
        next_size = next(Ns, -1)
        if next_size > 0:
            m = np.shape(b)[0] - 1
            R = make_restriction(m, next_size)
            P = make_interpolation(m, next_size)
            # print(np.shape(R), np.shape(A), np.shape(P))
            Ar = np.matmul(np.matmul(R, A), P)
            rr = np.matmul(R, r)
            er = v_cycle(Ar, np.zeros(next_size + 1), rr)
            e = np.matmul(P, er)
            u = u + e
            relax(A, u, b)
        return u
    
    return lambda u: v_cycle(A, u, f)
        

In [11]:
N = 25
A = np.diag(np.full(N, 10.)) + np.diag(np.full(N - 1, -1), -1) + np.diag(np.full(N - 1, -1), 1) 
b = np.zeros(25)

def gauss_jacobi_iter(A: np.array, b: np.array, x0: np.array, weight = 1):
    D = np.diag(np.diag(A))
    M = D
    prev = x0
    curr = prev + weight * np.matmul(LA.inv(M), b - np.matmul(A, prev))
    return curr

multigrid_v_cycle = make_multigrid_v_cycle_1d_iter(A, b, [8, 4], gauss_jacobi_iter)


1st V-Cycle:
[0.06581086 0.05126668 0.0367225  0.02217832 0.02118726 0.02019619
 0.01920513 0.0203547  0.02150427 0.02265385 0.02380342 0.02495299
 0.02610256 0.02495299 0.02380342 0.02265385 0.02150427 0.0203547
 0.01920513 0.02019619 0.02118726 0.02217832 0.0367225  0.05126668
 0.06581086]

2nd V-Cycle:
[0.00658109 0.00512667 0.00367225 0.00221783 0.00211873 0.00201962
 0.00192051 0.00203547 0.00215043 0.00226538 0.00238034 0.0024953
 0.00261026 0.0024953  0.00238034 0.00226538 0.00215043 0.00203547
 0.00192051 0.00201962 0.00211873 0.00221783 0.00367225 0.00512667
 0.00658109]


In [ ]:
u0 = np.ones(25)
u1 = multigrid_v_cycle(u0)
u2 = multigrid_v_cycle(u1)
print(f"1st V-Cycle:\n{u1}\n\n2nd V-Cycle:\n{u2}")

In [13]:
prev = u0
for _ in range(0, 2):
    u_gj = gauss_jacobi_iter(A, b, prev)
    prev = u_gj
    
print(u_gj)

[0.02 0.03 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.03 0.02]
